In [27]:
# Librerias utilizadas
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from standard_precip.spi import SPI
from standard_precip.utils import plot_index
from sklearn.preprocessing import MinMaxScaler

### Importación de datos

In [28]:
csv_names = ['DATOS_CUENCA_MEDITERRANEA_ANDALUZA', 'DATOS_CUENCA_GUADALQUIVIR', 'DATOS_CUENCA_GUADIANA',
                      'DATOS_CUENCA_SEGURA', 'DATOS_CUENCA_RIOS_GUADALETE_BARBATE', 'DATOS_CUENCA_RIOS_TINTO_ODIEL_PIEDRAS']
path = '..\\DATA\\'
keys_names = ['Demarcación Hidrográfica de las Cuencas Mediterráneas Andaluzas', 'Cuenca hidrográfica del Guadalquivir', 'Cuenca hidrográfica del Guadiana',
              'Cuenca hidrográfica del Segura', 'Demarcación Hidrográfica Guadalete-Barbate', 'Demarcación Hidrográfica Tinto, Odiel y Piedras']
df_dict = {key: pd.read_csv(path+value+'.csv', sep=';', encoding='ISO-8859-1')
           for key, value in zip(keys_names, csv_names)}


In [29]:
df_dict[keys_names[0]].head()

,Año,Mes,Precipitaciones,ISSP
0,1940,1,121.2300,1.195343
1,1940,2,54.0225,0.533104
2,1940,3,50.5694,0.602623
3,1940,4,11.0414,0.345051
4,1940,5,16.0715,0.223445


In [30]:
# Creación columna de fecha
def create_datetime_column(df):
    df['datetime'] = df.apply(lambda row: datetime(year=int(row['Año']), month=int(row['Mes']), day=1), axis=1)
    return df
for key, df in df_dict.items():
    create_datetime_column(df)

df_dict[keys_names[0]].head()

### Cáculo SPI

In [34]:
# Función que calcula el índice SPI
spi = SPI()
def calculate_spi(dataframe, escala_temporal):
    df_spi = spi.calculate(
        dataframe, 
        'datetime', 
        'Precipitaciones', 
        freq="M", 
        scale=escala_temporal, 
        fit_type="lmom", 
        dist_type="gam"
        )
    return df_spi['Precipitaciones_scale_'+str(escala_temporal)+'_calculated_index']

In [35]:
# Cálculo de spi_3, spi_6 y spi_12
for key, df in df_dict.items():
    df['spi_3'] = calculate_spi(df, 3)
    df['spi_6'] = calculate_spi(df, 6)
    df['spi_12'] = calculate_spi(df, 12)
    
df_dict[keys_names[0]].head(12)

### Limpieza y transformación

In [38]:
# Quitamos columnas, renombramos y ordenamos el dataframe
cols_to_remove = ['Año','Mes','Precipitaciones', 'Precipitaciones_scale_3', 'Precipitaciones_scale_6', 'Precipitaciones_scale_12']
for key, df in df_dict.items():
    df.drop(columns=cols_to_remove, inplace=True)
    df_dict[key] = df.rename(columns={'ISSP': 'IESP'})
#  Reordenamos en nuevo bucle para evitar problemas en los índices de colmunas
for key, df in df_dict.items():
    df_dict[key] = df.reindex(columns=['datetime', 'IESP', 'spi_3', 'spi_6', 'spi_12'])  

In [39]:
# Normalizacion de datos
scaler = MinMaxScaler()
for key, df in df_dict.items():
    df_dict[key][['IESP', 'spi_3', 'spi_6', 'spi_12']] = scaler.fit_transform(df_dict[key][['IESP', 'spi_3', 'spi_6', 'spi_12']])

df_dict[keys_names[0]].head(12)

### Exportación datos

In [40]:
# Datasets de salida
output_csv_names = ['df_DHCMA', 'df_CHG', 'df_CHGUADIANA',
            'df_CHS', 'df_DHGB', 'df_DHTOP']
key_list = list(df_dict.keys())
path = '..\\DATA\\'
for key, df in df_dict.items():
    df.to_csv(path + output_csv_names[key_list.index(key)] + '.csv', index=False)